## Задание 1

**Написать теггер на данных с руским языком**

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации.

2. 
   - написать свой теггер как на занятии, 
   - попробовать разные векторайзеры,
   - добавить знание не только букв но и слов
   
3. сравнить все реализованные методы сделать выводы

## загрузка данных

In [1]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

In [2]:
import pyconll

In [3]:
# from nltk.corpus import brown # Здесь, возможно необходимо будет взять только русские другую библиотеку.

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

# tags = [tag for (word, tag) in brown.tagged_words(categories='news')]
# nltk.FreqDist(tags)

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [6]:
# !wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
# !wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [7]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [8]:
# Здесь можно сократить написание.

In [9]:
train_data = []
for sent in full_train[:]:
    train_data.append([(token.form, token.upos) for token in sent])
  
test_data = []
for sent in full_test[:]:
    test_data.append([(token.form, token.upos) for token in sent])

test_sents=[]      
for sent in full_test[:]:
    test_sents.append([token.form for token in sent])

сгруппируем все теггеры и загоим их в одну функцию
или сделаем все методом перебора. 

#### 1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [10]:
default_tagger = nltk.DefaultTagger('NN')
default_tagger.evaluate(test_data)

0.0

In [11]:
unigram_tagger = UnigramTagger(train_data)
unigram_tagger.evaluate(test_data)

0.8772537323492737

In [12]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
bigram_tagger.evaluate(test_data)

0.8829828463586425

In [13]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
trigram_tagger.evaluate(test_data)

0.882081353418933

In [14]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data) 

0.8814747413473528

#### 2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

Попробуем еще разные варианты **векторайзеров**.


#### Построим модель работающую на символьных н-граммах.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [16]:
train_tok = []
train_label = []
for sent in train_data[:]:
    for tok in sent: # Вот здесь мы пройдемся по предложению с помощью окна.
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

In [17]:
test_tok = []
test_label = []
for sent in test_data[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [18]:
def get_tok_and_label(data):
    tokens = []
    labels = []
    for sent in data[:]:
        for tok in sent:
            tokens.append(tok[0])
            labels.append('NO_TAG' if tok[1] is None else tok[1])
    return tokens, labels

In [21]:
le = LabelEncoder()

train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [22]:
def cacl_accuracy(Vecorizer, Regression, ngram =(1,5), analyzer='char'):
    # count-base подход. подкручив 4,3 тыс.

    # if Vecorizer == HashingVectorizer:
    #     vectorizer = Vecorizer(ngram_range=ngram, analyzer=analyzer, n_features=100)
    # else:
    vectorizer = Vecorizer(ngram_range=ngram, analyzer=analyzer)

    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)

    r = Regression #(random_state=0)
    r.fit(X_train, train_enc_labels)
    
    pred = r.predict(X_test)
      
    accuracy = accuracy_score(test_enc_labels, pred)

    return accuracy

In [23]:
cacl_accuracy(HashingVectorizer, LogisticRegression(random_state=0), ngram=(1, 5))

0.9397600512250194

In [24]:
cacl_accuracy(HashingVectorizer, LogisticRegression(random_state=0), ngram=(2, 5))

0.8605129242071917

In [25]:
cacl_accuracy(CountVectorizer, LogisticRegression(random_state=0), ngram=(1, 5))

0.9532908704883227

In [26]:
cacl_accuracy(TfidfVectorizer, LogisticRegression(random_state=0), ngram=(1, 5))

0.9459356991204125

In [27]:
cacl_accuracy(CountVectorizer, LogisticRegression(random_state=0), ngram=(1, 2), analyzer='word')

0.7511373976342129

In [28]:
cacl_accuracy(TfidfVectorizer, LogisticRegression(random_state=0), ngram=(1, 2), analyzer='word')

0.7510699962929263

In [29]:
cacl_accuracy(CountVectorizer, xgb.XGBClassifier(verbosity=0), ngram=(1, 5))

0.9220840494725845

In [30]:
cacl_accuracy(TfidfVectorizer, xgb.XGBClassifier(verbosity=0), ngram=(1, 5))

0.9278468641525967

## Результат. 
 Проверили разные векторайзеры, для разного к-ва символов. По итогу оказалось, что символьные N-граммы показали результат лучше чем N-граммы на словах.

# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
  * передаём в сетку токен и его соседей
  * передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [ ]:
# !pip install nltk
# !pip install corus
# !pip install razdel
#!pip install -U spacy
#!python -m spacy info

In [1]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
import pyconll
import corus
from corus import load_ne5
import re
import spacy
from spacy import displacy
import ru_core_news_md

%matplotlib inline

In [2]:
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [ ]:
# !wget http://www.labinform.ru/pub/named_entities/collection5.zip

# !unzip collection5.zip

# !rm collection5.zip

# !ls

In [7]:
dir = 'Collection5/'
records = load_ne5(dir)
# next(records) # Необходимо каким-то образоб обработать все тексты.

процедуры обработки взять из вебинарного ноутбука

In [8]:
document = next(records)
text = document.text
# text
text = re.sub('\r\n\r\n',' ',text)
text = re.sub('\r\n',' ',text)
text

'Обама назначил своего нового советника по экономическим вопросам Барак Обама в пятницу, 7 января, официально объявил о назначении нового директора Национального экономического совета администрации президента. Как передает Associated Press, им стал советник министра финансов США Джин Сперлинг (Gene Sperling). По новой должности чиновник станет главным советником Обамы по экономическим вопросам и ключевой фигурой в формировании экономической политики американского президента, влияя, в частности, на формирование федерального бюджета. Сперлинг уже возглавлял Национальный экономический совет в в 1996-2000 годах, при администрации Билла Клинтона. AP отмечает, что назначение Сперлинга стало частью перестановок в команде Обамы, последовавших за прошедшими осенью 2010 года промежуточными выборами в Конгресс, в ходе которых Республиканской партии удалось существенно укрепить свои позиции. Сообщается также, что, по мнению чиновников из президентской администрации, Сперлинг произвел на Обаму боль

In [9]:
document.text = text

In [10]:
nltk.pos_tag(nltk.word_tokenize(text)) # Как предварительно очистить все статьи в словаре.

[('Обама', 'JJ'),
 ('назначил', 'NNP'),
 ('своего', 'NNP'),
 ('нового', 'NNP'),
 ('советника', 'NNP'),
 ('по', 'NNP'),
 ('экономическим', 'NNP'),
 ('вопросам', 'NNP'),
 ('Барак', 'NNP'),
 ('Обама', 'NNP'),
 ('в', 'NNP'),
 ('пятницу', 'NNP'),
 (',', ','),
 ('7', 'CD'),
 ('января', 'NN'),
 (',', ','),
 ('официально', 'NNP'),
 ('объявил', 'NNP'),
 ('о', 'NNP'),
 ('назначении', 'NNP'),
 ('нового', 'NNP'),
 ('директора', 'NNP'),
 ('Национального', 'NNP'),
 ('экономического', 'NNP'),
 ('совета', 'NNP'),
 ('администрации', 'NNP'),
 ('президента', 'NNP'),
 ('.', '.'),
 ('Как', 'NN'),
 ('передает', 'NN'),
 ('Associated', 'NNP'),
 ('Press', 'NNP'),
 (',', ','),
 ('им', 'NNP'),
 ('стал', 'NNP'),
 ('советник', 'NNP'),
 ('министра', 'NNP'),
 ('финансов', 'NNP'),
 ('США', 'NNP'),
 ('Джин', 'NNP'),
 ('Сперлинг', 'NNP'),
 ('(', '('),
 ('Gene', 'NNP'),
 ('Sperling', 'NNP'),
 (')', ')'),
 ('.', '.'),
 ('По', 'JJ'),
 ('новой', 'JJ'),
 ('должности', 'NN'),
 ('чиновник', 'NNP'),
 ('станет', 'NNP'),
 ('глав

In [11]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) if hasattr(chunk, 'label') }

{('AP', 'ORGANIZATION'),
 ('Associated Press', 'ORGANIZATION'),
 ('Gene', 'ORGANIZATION'),
 ('Larry Summers', 'PERSON'),
 ('William Daley', 'PERSON'),
 ('Барак Обама', 'PERSON'),
 ('Днем', 'PERSON'),
 ('Обама', 'PERSON'),
 ('Роберт Гиббс', 'PERSON'),
 ('Сперлинг', 'PERSON'),
 ('Уильям Дэйли', 'PERSON')}

In [12]:
document.spans

[Ne5Span(
     index='T1',
     type='PER',
     start=0,
     stop=5,
     text='Обама'
 ),
 Ne5Span(
     index='T2',
     type='PER',
     start=68,
     stop=79,
     text='Барак Обама'
 ),
 Ne5Span(
     index='T3',
     type='ORG',
     start=150,
     stop=185,
     text='Национального экономического совета'
 ),
 Ne5Span(
     index='T4',
     type='MEDIA',
     start=225,
     stop=241,
     text='Associated Press'
 ),
 Ne5Span(
     index='T5',
     type='GEOPOLIT',
     start=278,
     stop=281,
     text='США'
 ),
 Ne5Span(
     index='T6',
     type='PER',
     start=282,
     stop=311,
     text='Джин Сперлинг (Gene Sperling)'
 ),
 Ne5Span(
     index='T7',
     type='PER',
     start=370,
     stop=375,
     text='Обамы'
 ),
 Ne5Span(
     index='T8',
     type='PER',
     start=546,
     stop=554,
     text='Сперлинг'
 ),
 Ne5Span(
     index='T9',
     type='ORG',
     start=570,
     stop=602,
     text='Национальный экономический совет'
 ),
 Ne5Span(
     index='T10',

In [22]:
# displacy.render(text, jupyter=True, style='ent')

### Spacy

In [ ]:
# !pip install -U spacy
# !python -m spacy info

In [ ]:
# !python -m spacy download ru_core_news_sm

In [13]:
#!pip install -U spacy
#!python -m spacy info
import spacy
from spacy import displacy
import ru_core_news_md

In [24]:
import spacy
from spacy.lang.ru.examples import sentences 

nlp = spacy.load("ru_core_news_sm")

In [25]:
from spacy.lang.ru import Russian

In [26]:
# nlp = ru_core_news_md.load()
nlp = spacy.load("ru_core_news_sm")
ny_bb = text #url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)

In [28]:
displacy.render(article, jupyter=True, style='ent')

In [ ]:
# установка deeppavlov

# !pip uninstall -y tensorflow tensorflow-gpu
# !pip install numpy scipy librosa unidecode inflect librosa transformers
# !pip install deeppavlov

https://files.deeppavlov.ai/0.16/ner/ner_rus_bert_torch.tar.gz

In [ ]:
# !python -m deeppavlov install squad_bert
# !python -m deeppavlov install ner_ontonotes

In [ ]:
# !activate deeppavlov_env
# !python -m spacy download en
# !python -m spacy download en_core_web_sm

In [ ]:
# !pip install razdel
# !pip install numpy==1.19.5
# !pip install fasttext

In [5]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=False)

In [6]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from razdel import tokenize
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score

In [17]:
rus_document = text #"Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

[[['Обама',
   'назначил',
   'своего',
   'нового',
   'советника',
   'по',
   'экономическим',
   'вопросам',
   'Барак',
   'Обама',
   'в',
   'пятницу',
   ',',
   '7',
   'января',
   ',',
   'официально',
   'объявил',
   'о',
   'назначении',
   'нового',
   'директора',
   'Национального',
   'экономического',
   'совета',
   'администрации',
   'президента',
   '.',
   'Как',
   'передает',
   'Associated',
   'Press',
   ',',
   'им',
   'стал',
   'советник',
   'министра',
   'финансов',
   'США',
   'Джин',
   'Сперлинг',
   '(',
   'Gene',
   'Sperling',
   ')',
   '.',
   'По',
   'новой',
   'должности',
   'чиновник',
   'станет',
   'главным',
   'советником',
   'Обамы',
   'по',
   'экономическим',
   'вопросам',
   'и',
   'ключевой',
   'фигурой',
   'в',
   'формировании',
   'экономической',
   'политики',
   'американского',
   'президента',
   ',',
   'влияя',
   ',',
   'в',
   'частности',
   ',',
   'на',
   'формирование',
   'федерального',
   'бюджета'

In [58]:
deeppavlov_ner(['Example sentence'])

[[['Example', 'sentence']],
 array([[[0.92888683, 0.01832325, 0.05278989],
         [0.39169678, 0.02965371, 0.5786495 ]]], dtype=float32)]

In [18]:
tokens, tags = deeppavlov_ner(['Саша живет в Нижнем Новгороде'])
for tok, tag in zip(tokens[0], tags[0]):
    print(f'{tok}\t{tag}')

Саша	[0.05651835 0.94057846 0.00290323]
живет	[9.9798691e-01 1.7001238e-03 3.1296091e-04]
в	[9.9913329e-01 7.7816914e-04 8.8671215e-05]
Нижнем	[0.2384246  0.7537949  0.00778052]
Новгороде	[0.20955132 0.7804916  0.00995718]


### Загрузим корпус слов. И вытащим из него все теги.

In [7]:
dir = 'Collection5/'
records = load_ne5(dir)

words_docs = []
for ix, rec in enumerate(records):
    words = []
    text = rec.text
    for token in tokenize(text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        token.text = re.sub("[^a-zA-Z0-9а-яА-Я]","", token.text)
        if (token.text == '') or (len(token.text) < 3):
            continue
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [8]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [9]:
df_words['tag'].value_counts()

OUT         142056
PER          17266
ORG          11525
LOC           4503
GEOPOLIT      3562
MEDIA         2074
Name: tag, dtype: int64

### Проделаем аналогичную операцию на библиотеке deeppavlov.

In [ ]:
from deeppavlov import configs, build_model

ner = build_model(configs.ner.ner_rus, download=False)

In [11]:
tokens, tags = ner(['Саша живет в Нижнем Новгороде'])
for tok, tag in zip(tokens[0], tags[0]):
    print(f'{tok}\t{tag}')

Саша	B-PER
живет	O
в	O
Нижнем	B-LOC
Новгороде	I-LOC


In [12]:
dir = 'Collection5/'
records = load_ne5(dir)

words_docs = []
for ix, rec in tqdm(enumerate(records)):
    words = []
    text = rec.text
    tokens, tags = ner([text])
    for token, tag in zip(tokens[0], tags[0]):
        tag = re.sub("B-","",tag)
        tag = re.sub("I-","",tag)
        if tag == 'O':
            tag = 'OUT'

        token = re.sub("[^a-zA-Z0-9а-яА-Я]","", token)
        if (token == '') or (len(token) < 3):
            continue
        
        words.append([token, tag])     
    words_docs.extend(words)

1000it [01:28, 11.31it/s]


In [13]:
deeppavlov_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [14]:
deeppavlov_words['tag'].value_counts()

OUT    141468
PER     17216
ORG     13307
LOC      8908
Name: tag, dtype: int64

In [15]:
df_words['tag'].value_counts()

OUT         142056
PER          17266
ORG          11525
LOC           4503
GEOPOLIT      3562
MEDIA         2074
Name: tag, dtype: int64

In [16]:
len(deeppavlov_words), len(df_words)

(180899, 180986)

Как видно наша предобученная сеть из DeepPavlov не распознает сущности GEOPOLIT и MEDIA.

Судя по всему DeepPavlov выполняет какую-то дополнительную предобработку текста. 
т.к. к-во токенов после обработки в данной библиотеке уменьшилось относительно осходного значения.
 Из-за такого несоответствия в названиях токенов и том что к-во записей в результирующих таблицах отличаются вычесление к-ва предсказания становиться не тривиальной задачей.

In [28]:
df_words_1 = df_words

df_words_1['tag_pred']=''

In [ ]:
i = 0
l = len(df_words_1)
deep_len = len(deeppavlov_words)
for j in tqdm(range(len(df_words_1))):
    # print(df_words_1.iloc[j].word, deeppavlov_words.iloc[i].word)
    # if df_words_1.iloc[j].word == deeppavlov_words.iloc[j].word:
    if (re.findall(df_words_1.iloc[j].word,deeppavlov_words.iloc[i].word) !=[]) or (re.findall(deeppavlov_words.iloc[i].word, df_words_1.iloc[j].word) !=[]):
        df_words_1['tag_pred'][j] = deeppavlov_words.iloc[i].tag
        i+=1
    else:
        print(df_words_1.iloc[j].word, deeppavlov_words.iloc[j].word, df_words_1.iloc[j].tag)
    
    if i > deep_len:
        break

In [34]:
df_words_1.loc[(df_words_1['tag_pred']=='')]

,word,tag,tag_pred
3111,press,MEDIA,
3214,выдвинули,OUT,
3215,иные,OUT,
3216,версии,OUT,
3217,отставки,OUT,
...,...,...,...
180981,проблемы,OUT,
180982,оперативном,OUT,
180983,порядке,OUT,
180984,заявил,OUT,


Вот и наши различия. С какого-то момента алгоритм сравнения затыкается и начинает пропускать всеподряд.

In [ ]:
#================================================================================================#
#================================================================================================#

__2.__  написать свой нер попробовать разные подходы
    - передаём в сетку токен и его соседей
    - передаём в сетку только токен

# ! Внимание
   * Для запуска следующего кода потребутся Tensorflow версии **> 2.0**. Поэтому этот участок кода лучше всего запускать в виртуально окружении отличном от того в котором установлена библиотека **Deeppavlov**.

In [ ]:
# Попробуем нейронную сеть.

In [ ]:
# !wget http://www.labinform.ru/pub/named_entities/collection5.zip

# !unzip collection5.zip
# !pip install razdel

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from razdel import tokenize
from corus import load_ne5

In [3]:
def get_classification_report(y_test_true, y_test_pred):
    print(classification_report(y_test_true, y_test_pred))

    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

Загрузим размеченный корпус текстов.

In [4]:
dir = 'Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='146',
    text='Обама назначил своего нового советника по экономическим вопросам\r\n\r\nБарак Обама в пятницу, 7 января, официально объявил о назначении нового директора Национального экономического совета администрации президента. Как передает Associated Press, им стал советник министра финансов США Джин Сперлинг (Gene Sperling).\r\n\r\nПо новой должности чиновник станет главным советником Обамы по экономическим вопросам и ключевой фигурой в формировании экономической политики американского президента, влияя, в частности, на формирование федерального бюджета.\r\n\r\nСперлинг уже возглавлял Национальный экономический совет в в 1996-2000 годах, при администрации Билла Клинтона.\r\n\r\nAP отмечает, что назначение Сперлинга стало частью перестановок в команде Обамы, последовавших за прошедшими осенью 2010 года промежуточными выборами в Конгресс, в ходе которых Республиканской партии удалось существенно укрепить свои позиции.\r\n\r\nСообщается также, что, по мнению чинов

In [5]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [6]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [7]:
df_words['tag'].value_counts()

OUT         219021
PER          21165
ORG          13642
LOC           4568
GEOPOLIT      4355
MEDIA         2479
Name: tag, dtype: int64

In [8]:
df_words.head(6)

,word,tag
0,Адвокат,OUT
1,Мурадвердиева,PER
2,опровергает,OUT
3,обвинения,OUT
4,в,OUT
5,контрабанде,OUT


In [9]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [10]:
encoder.classes_

array(['GEOPOLIT', 'LOC', 'MEDIA', 'ORG', 'OUT', 'PER'], dtype=object)

In [11]:
train_x.apply(len).max(axis=0)

32

In [12]:
valid_x

231283    Министерство
16162           китаец
134288               )
43696          которой
149725             две
              ...     
222463       Гиоргадзе
79942               !!
249693       студентов
82347         комитета
50903         военными
Name: word, Length: 66308, dtype: object

In [13]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [14]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста.
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(  
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            #ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [16]:
len(vectorize_layer.get_vocabulary())

29851

In [17]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax') # [OUT, PER, ORG, LOC, GEOPOLIT, MEDIA]

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [18]:
mmodel = modelNER()

In [19]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [20]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12433/12433 [==============================] - 759s 60ms/step - loss: 0.2928 - accuracy: 0.9142 - val_loss: 0.2101 - val_accuracy: 0.9376
Epoch 2/3
12433/12433 [==============================] - 678s 55ms/step - loss: 0.1252 - accuracy: 0.9628 - val_loss: 0.3859 - val_accuracy: 0.8917
Epoch 3/3
12433/12433 [==============================] - 581s 47ms/step - loss: 0.1090 - accuracy: 0.9656 - val_loss: 0.3098 - val_accuracy: 0.8935


In [21]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

In [22]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.9008850878035005

In [23]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1093
           1       0.86      0.77      0.81      1100
           2       0.93      0.81      0.87       628
           3       0.86      0.56      0.68      3428
           4       0.97      0.92      0.94     54894
           5       0.49      0.88      0.63      5165

    accuracy                           0.89     66308
   macro avg       0.84      0.81      0.81     66308
weighted avg       0.92      0.89      0.90     66308

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      987   21    1    30     14    40
1       14  843    2    24     34   183
2        3    5  507    12     56    45
3       68   46   23  1925   1007   359
4        5   65   10   239  50422  4153
5        1    0    1    12    586  4565


### Обучим нейронную сеть на биграммах и триграммах. 

In [25]:
def custom_standardization(input_data):
    # Здесь может быть предобработка текста.
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization( 
                            standardize=custom_standardization,
                            max_tokens=vocab_size,
                            output_mode='int',
                            ngrams=(1, 3),
                            output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [26]:
mmodel = modelNER()

In [27]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [28]:
mmodel.fit( train_data,
            validation_data=valid_data,
            epochs=3)

Epoch 1/3
12433/12433 [==============================] - 359s 29ms/step - loss: 0.2928 - accuracy: 0.9148 - val_loss: 0.2099 - val_accuracy: 0.9375
Epoch 2/3
12433/12433 [==============================] - 327s 26ms/step - loss: 0.1254 - accuracy: 0.9627 - val_loss: 0.4082 - val_accuracy: 0.8911
Epoch 3/3
12433/12433 [==============================] - 339s 27ms/step - loss: 0.1098 - accuracy: 0.9654 - val_loss: 0.2455 - val_accuracy: 0.8926


In [29]:
pred_y = mmodel.predict(valid_x)
y_pred_classes = np.argmax(pred_y,axis=1)

In [30]:
f1 = f1_score(valid_y, y_pred_classes, average= "weighted")
f1

0.900048139443574

In [31]:
print(f"Classes: {encoder.classes_}\r\n")

get_classification_report(valid_y, y_pred_classes)

Classes: ['GEOPOLIT' 'LOC' 'MEDIA' 'ORG' 'OUT' 'PER']

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1093
           1       0.87      0.75      0.81      1100
           2       0.92      0.81      0.86       628
           3       0.83      0.57      0.67      3428
           4       0.97      0.92      0.94     54894
           5       0.49      0.88      0.63      5165

    accuracy                           0.89     66308
   macro avg       0.83      0.81      0.80     66308
weighted avg       0.92      0.89      0.90     66308

CONFUSION MATRIX

col_0    0    1    2     3      4     5
row_0                                  
0      987   20    1    31     14    40
1       12  828    7    20     50   183
2        3    5  507    12     56    45
3       69   47   25  1940    988   359
4        5   54   10   312  50361  4152
5        1    0    1    12    586  4565


In [32]:
# TODO #1 Организуем обучение нейронной сети на символьных N-граммах. Пока не до конца понятно как их подавать в слой!